# 00. Imports and env variables

In [172]:
from transformers import pipeline

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain_community.llms import HuggingFacePipeline
import torch
from huggingface_hub import InferenceClient


from pydantic import BaseModel
from typing import List, Optional

import os
from dotenv import load_dotenv

import json

load_dotenv()

True

In [173]:
token = os.getenv("HF_TOKEN")

# 01. Problem Definition

In [174]:
# parser classes

# gaps
class Gap(BaseModel):
    title: str
    description: str
    impact: str
    actions: List[str]

class GapAnalysis(BaseModel):
    company_name: str
    job_title: str
    generation_date: str
    gaps: List[Gap]

# resume
class Header(BaseModel):
    name: str
    title: str
    location: str
    phone: str
    email: str
    linkedin: str
    github: str
    kaggle: Optional[str] = None

Bullet = str

class ProfessionalExperience(BaseModel):
    company: str
    period: str
    role: str
    location: str
    bullets: List[Bullet]

class Education(BaseModel):
    institution: str
    location: str
    degree: str
    period: str

class Project(BaseModel):
    name: str
    year: str
    context: str
    bullets: List[Bullet]
    github: Optional[str] = None

class Activity(BaseModel):
    organization: str
    period: str
    role: str
    location: str

class Certification(BaseModel):
    name: str
    issuer: str
    year: str

class SkillGroup(BaseModel):
    label: str
    entries: list[str]

class SkillsSection(BaseModel):
    groups: List[SkillGroup]

class Resume(BaseModel):
    header: Header
    experience: List[ProfessionalExperience]
    education: Optional[List[Education]] = None
    projects: Optional[List[Project]] = None
    activities: Optional[List[Activity]] = None
    certifications: Optional[List[Certification]] = None
    skills: Optional[SkillsSection] = None

In [175]:
gaps_schema = GapAnalysis.model_json_schema()

resume_schema = Resume.model_json_schema()

In [176]:
# model for inference

client = InferenceClient(
    model="Qwen/Qwen2.5-7B-Instruct",
    token=token  # ou via env
)


In [177]:
# prompts

with open('./00_prompts/gaps.txt', 'r') as f:
    gaps_prompt = f.read()

with open('./00_prompts/resume.txt', 'r') as f:
    resume_prompt = f.read()

In [178]:
# task prompt

gaps_prompt_template = PromptTemplate(
    input_variables=["job_description", "profile"],
    template=gaps_prompt
)

resume_prompt_template = PromptTemplate(
    input_variables=["job_description", "profile"],
    template=resume_prompt
)

In [179]:
# getting descriptions

with open('./01_jobs/job_1.txt', 'r') as f:
    job_description = f.read()

with open('./02_source/profile.txt', 'r') as f:
    profile = f.read()

In [180]:
# prompt filling

formatted_gaps_prompt = gaps_prompt_template.format(
    job_description=job_description,
    profile=profile,
    schema=gaps_schema
)

formatted_resume_prompt = resume_prompt_template.format(
    job_description=job_description,
    profile=profile,
    schema=resume_schema
)

# 02. LLM execution and post processing

In [181]:
formatted_resume_prompt

"Você é um especialista em RH e construção de currículos.\n\nTarefa:\nAnalise a vaga e o perfil do candidato e gere um currículo com o maior match possível usando as informações do candidato (mude a forma de escrita, mas não omita nenhuma experiencia profissional ou formação academica).\n\nVaga:\nDescrição da vaga\n\nBuscamos profissional com especialidade em aplicar Inteligência Artificial Generativa para integrar nossa equipe de projetos voltados ao atendimento ao cliente. Como integrante do time, será responsável por desenvolver e implementar globalmente soluções inovadoras utilizando principalmente tecnologias de IA generativa, contribuindo diretamente para otimizar nosso negócio e transformar a experiência do cliente através de soluções tecnológicas avançadas.\n\nResponsabilidades e atribuições\n\nDesenvolver de ponta a ponta soluções de negócio utilizando tecnologias de IA generativa e aprendizado de máquina;\nUtilizar ferramentas low/no-code como aceleradores do desenvolvimento 

In [182]:
# output generation and parsing

gaps_output = client.chat_completion(messages=[{"role": "user", "content": formatted_gaps_prompt}])
gaps_parsed = GapAnalysis.model_validate_json(gaps_output.choices[0].message.content)
gaps = gaps_parsed.model_dump(mode="python")

resume_output = client.chat_completion(messages=[{"role": "user", "content": formatted_resume_prompt}])
resume_parsed = Resume.model_validate_json(resume_output.choices[0].message.content)
resume = resume_parsed.model_dump(mode="python")

# 03. Storing output

In [183]:
# storing output

with open('./03_jsons/gap_analysis.json', 'w') as f:
    json.dump(gaps, f, indent=2, ensure_ascii=False)

with open('./03_jsons/resume.json', 'w') as f:
    json.dump(resume, f, indent=2, ensure_ascii=False)